https://adventofcode.com/2022/day/23

In [1]:
from collections import deque, defaultdict
from itertools import product

In [2]:
with open("data/23.txt") as fh:
    data = fh.read()

In [3]:
def load_data(data):
    D = {}
    for yminus, line in enumerate(data.strip().splitlines()):
        for x, c in enumerate(line):
            if c == "#":
                D[complex(x, -yminus)] = None
    return D

In [4]:
testdata = """\
....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..
"""

In [5]:
test_crater = load_data(testdata)
len(test_crater)

22

In [6]:
sectors_tpl = (
    (0+1j, 1+1j, -1+1j),
    (0-1j, 1-1j, -1-1j),
    (-1, -1+1j, -1-1j),
    (1, 1+1j, 1-1j)
)

In [7]:
drxns = [
    1j,
    1+1j,
    1,
    1-1j,
    -1j,
    -1-1j,
    -1,
    -1+1j
]

In [8]:
def diffuse(crater, rounds):
    sectors = deque(list(sectors_tpl))
    for i in range(rounds):
        step(crater, sectors)
        sectors.append(sectors.popleft())
        # print(i+1)
        # print_crater(crater)
        # print()


def step(D, sectors):
    for elf in D:
        if not any((elf + drxn in D) for drxn in drxns):
            continue
        for sec in sectors:
            if not any((elf + drxn in D) for drxn in sec):
                D[elf] = sec[0]
                break
    targets = defaultdict(list)
    for k, v in D.items():
        if v is not None:
            targets[k+v].append(k)
    for k, v in targets.items():
        if len(v) == 1:
            del D[v[0]]
            D[k] = None
    for k in D:
        D[k] = None


def count_empty(D):
    xmin = int(min(p.real for p in D))
    xmax = int(max(p.real for p in D))
    ymin = int(min(p.imag for p in D))
    ymax = int(max(p.imag for p in D))
    return sum(complex(x, y) not in D for x in range(xmin, xmax+1) for y in range(ymin, ymax+1))


def print_crater(D):
    for y in range(int(max(p.imag for p in D)), int(min(p.imag for p in D)) - 1, -1):
        print("".join('#' if complex(x, y) in D else '.' for x in
                      range(int(min(p.real for p in D)), int(max(p.real for p in D)) + 1)))

In [9]:
test_crater = load_data(testdata)
diffuse(test_crater, 10)
count_empty(test_crater)

110

In [10]:
%%time
crater = load_data(data)
diffuse(crater, 10)
count_empty(crater)

CPU times: user 107 ms, sys: 0 ns, total: 107 ms
Wall time: 106 ms


3925

Part 2

In [11]:
def diffuse2(crater, maxrounds=1000):
    sectors = deque(list(sectors_tpl))
    for i in range(1, maxrounds):
        moved = step2(crater, sectors)
        if not moved:
            break
        sectors.append(sectors.popleft())
    return i, moved


def step2(D, sectors):
    for elf in D:
        if not any((elf + drxn in D) for drxn in drxns):
            continue
        for sec in sectors:
            if not any((elf + drxn in D) for drxn in sec):
                D[elf] = sec[0]
                break
    targets = defaultdict(list)
    for k, v in D.items():
        if v is not None:
            targets[k+v].append(k)
    moved = 0
    for k, v in targets.items():
        if len(v) == 1:
            del D[v[0]]
            D[k] = None
            moved += 1
    for k in D:
        D[k] = None
    return moved



In [12]:
%%time
test_crater = load_data(testdata)
diffuse2(test_crater)

CPU times: user 1.18 ms, sys: 0 ns, total: 1.18 ms
Wall time: 1.18 ms


(20, 0)

In [13]:
%%time
crater = load_data(data)
diffuse2(crater)

CPU times: user 4.7 s, sys: 0 ns, total: 4.7 s
Wall time: 4.73 s


(903, 0)